In [1]:
from pyscf import scf,gto,cc
import numpy as np,numpy
from FcMole import FcM

In [2]:
def DeltaV(mol,dL): # dL=[[atmidxs],[atmFcs]]
    mol.set_rinv_orig_(mol.atom_coords()[dL[0][0]])
    dV=mol.intor('int1e_rinv')*dL[1][0]
    for i in range(1,len(dL[0])): 
        mol.set_rinv_orig_(mol.atom_coords()[dL[0][i]])
        dV+=mol.intor('int1e_rinv')*dL[1][i]
    return -dV

In [3]:
#coefficients accurate dL^-8
d1=np.asarray([1/280,-4/105,1/5,-4/5,0,4/5,-1/5,4/105,-1/280])
d2=np.asarray([-1/560,8/315,-1/5,8/5,-205/72,8/5,-1/5,8/315,-1/560])
d3=np.asarray([-7/240,3/10,-169/120,61/30,0,-61/30,169/120,-3/10,7/240])
d4=np.asarray([7/240,-2/5,169/60,-122/15,91/8,-122/15,169/60,-2/5,7/240])
d5=np.asarray([1/6,-3/2,13/3,-29/6,0,29/6,-13/3,3/2,-1/6])
d6=np.asarray([-1/4,3,-13,29,-75/2,29,-13,3,-1/4])
coeffs=np.array([d1,d2,d3,d4,d5,d6])

In [10]:
dl=.1
Ps=[]
Es=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e())
    ccalc=cc.CCSD(mf)
    ccalc.run()
    Es.append(ccalc.e_tot-mf.energy_nuc())
    Ps.append(np.einsum('pi,ij,qj->pq', mf.mo_coeff, ccalc.make_rdm1(), mf.mo_coeff))

converged SCF energy = -114.377014116702
E(CCSD) = -114.4652400334409  E_corr = -0.08822591673875277
converged SCF energy = -113.545536924897
E(CCSD) = -113.6430803803882  E_corr = -0.09754345549124638
converged SCF energy = -112.742145890619
E(CCSD) = -112.849666107315  E_corr = -0.1075202166956961
converged SCF energy = -111.967431228079
E(CCSD) = -112.0851825845285  E_corr = -0.1177513564492804
converged SCF energy = -111.221914650137
E(CCSD) = -111.3497197542642  E_corr = -0.1278051041274061
converged SCF energy = -110.506048710199
E(CCSD) = -110.6432890073352  E_corr = -0.1372402971359631
converged SCF energy = -109.820219990832
E(CCSD) = -109.9658441993197  E_corr = -0.1456242084872498
converged SCF energy = -109.164754557585
E(CCSD) = -109.3173091948976  E_corr = -0.15255463731213
converged SCF energy = -108.539924655925
E(CCSD) = -108.697611274585  E_corr = -0.1576866186598762


In [11]:
mol0=gto.M(atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
dV=DeltaV(mol0,[[0,1],[1,-1]])
mf0=scf.RHF(mol0)
mf0.scf()
cc0=cc.CCSD(mf0)
cc0.run()
E0=cc0.e_tot
P0=np.einsum('pi,ij,qj->pq', mf0.mo_coeff, cc0.make_rdm1(), mf0.mo_coeff.conj())

converged SCF energy = -111.221914650137
E(CCSD) = -111.349719744743  E_corr = -0.1278050946057308


In [13]:
molT=FcM(fcs=[1,-1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
mfT=scf.RHF(molT)
mfT.scf(dm0=mf.init_guess_by_1e())
ccT=cc.CCSD(mfT)
ccT.run()
ET=ccT.e_tot
PT=np.einsum('pi,ij,qj->pq', mfT.mo_coeff, ccT.make_rdm1(), mfT.mo_coeff)

converged SCF energy = -105.444757226602
E(CCSD) = -105.5894764428001  E_corr = -0.1447192161981249


In [14]:
np.einsum('ij,ij',P0,dV),np.einsum('ij,ij',mf0.make_rdm1(),dV)

(6.235764204299627, 6.355230016182793)

In [15]:
#dE/dZ^(n+1) = \int dP/dZ^n * dV
e=E0+np.einsum('ij,ij',P0,dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e,np.einsum('ij,ij',P0,dV) )
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(dl)**(i+1))
    e+=np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,dV)/np.math.factorial(i+2))

-104.6377650642529 6.235764204299627
-105.60824588048727 -0.9704808162343683
-105.58462078536466 0.023625095122604505
-105.56339054697425 0.021230238390414463
-105.59157979387707 -0.02818924690283286
-105.5961648374193 -0.004585043542227945
-105.59845390419638 -0.0022890667770754714


In [16]:
#expanding energies
e=E0+(mfT.energy_nuc()-mf0.energy_nuc())
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/dl**(i+1))

-104.61647932819768 6.2570499403548405
-105.5921366483776 -0.9756573201799205
-105.58823721172381 0.0038994366537981042
-105.56377373457775 0.024463477146056487
-105.5837680005125 -0.01999426593476035
-105.59389541651461 -0.010127416002111048


In [19]:
#to BF
molT=FcM(fcs=[-1,1],atom="C 0 0 0;O 0 0 2.1", unit="Bohr" )
mfT=scf.RHF(molT)
mfT.scf(dm0=mf.init_guess_by_1e())
ccT=cc.CCSD(mfT)
ccT.run()
ET=ccT.e_tot
PT=np.einsum('pi,ij,qj->pq', mfT.mo_coeff, ccT.make_rdm1(), mfT.mo_coeff.conj())

converged SCF energy = -119.917957127279
E(CCSD) = -119.9803459480259  E_corr = -0.06238882074723519


In [20]:
e=E0+np.einsum('ij,ij',P0,-dV) +(mfT.energy_nuc()-mf0.energy_nuc())
print(e)
for i in range (6):
    dP=np.einsum('xij,x->ij',Ps,coeffs[i]/(-dl)**(i+1))    # -dl to invert the direction
    e+=np.einsum('ij,ij',dP,-dV)/np.math.factorial(i+2)
    print(e,np.einsum('ij,ij',dP,-dV)/np.math.factorial(i+2))

-119.01405537761407
-119.98453619384844 -0.9704808162343683
-120.00816128897105 -0.023625095122604505
-119.98693105058064 0.021230238390414463
-119.95874180367781 0.02818924690283286
-119.96332684722003 -0.004585043542227945
-119.96103778044295 0.0022890667770754714


In [21]:
#expanding energies
e=E0
for i in range (6):
    e+=np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/(-dl)**(i+1)
    print(e,np.sum(coeffs[i]*Es)/np.math.factorial(i+1)/(-dl)**(i+1))

-117.60676968509786 -6.2570499403548405
-118.58242700527778 -0.9756573201799205
-118.58632644193158 -0.0038994366537981042
-118.56186296478552 0.024463477146056487
-118.54186869885076 0.01999426593476035
-118.55199611485287 -0.010127416002111048
